#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter ITP Audit Dashboard ( 2020 ) Parameters
Dashboard that shows performance metrics across browser to see the impact of ITP.
 1. Follow the instructions from <a href="https://docs.google.com/document/d/1HaRCMaBBEo0tSKwnofWNtaPjlW0ORcVHVwIRabct4fY/edit?usp=sharing" target="_blank">this document</a>
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'recipe_name': '',  # Name of document to deploy to.
  'auth_read': 'user',  # Credentials used for reading data.
  'auth_write': 'service',  # Credentials used for writing data.
  'recipe_slug': 'ITP_Audit_Dashboard',  # BigQuery dataset for store dashboard tables.
  'cm_account_id': '',  # Campaign Manager Account Id.
  'cm_advertiser_ids': '',  # Optional: Comma delimited list of CM advertiser ids.
  'floodlight_configuration_id': '',  # Floodlight Configuration Id for the Campaign Manager floodlight report.
  'dv360_partner_ids': '',  # Comma delimited list of DV360 Partner ids.
  'dv360_advertiser_ids': '',  # Optional: Comma delimited list of DV360 Advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute ITP Audit Dashboard ( 2020 )
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'drive': {
      'hour': [
      ],
      'auth': 'user',
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/1rH_PGXOYW2mVdmAYnKbv6kcaB6lQihAyMsGtFfinnqg/',
        'destination': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
      }
    }
  },
  {
    'dataset': {
      'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
      'auth': 'user'
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_PARTNER': {
            'values': {'field': {'order': 5,'name': 'dv360_partner_ids','default': '','kind': 'integer_list','description': 'Comma delimited list of DV360 Partner ids.'}}
          },
          'FILTER_ADVERTISER': {
            'values': {'field': {'order': 6,'name': 'dv360_advertiser_ids','default': '','kind': 'integer_list','description': 'Optional: Comma delimited list of DV360 Advertiser ids.'}}
          }
        },
        'timeout': 90,
        'body': {
          'timezoneCode': {'field': {'default': 'America/Los_Angeles','name': 'recipe_timezone','description': 'Timezone for report dates.','kind': 'timezone'}},
          'metadata': {
            'format': 'CSV',
            'title': {'field': {'prefix': 'ITP_Audit_Browser_','name': 'recipe_name','order': 1,'kind': 'string','description': 'Name of report in DV360, should be unique.'}},
            'dataRange': 'LAST_365_DAYS'
          },
          'params': {
            'metrics': [
              'METRIC_MEDIA_COST_ADVERTISER',
              'METRIC_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS',
              'METRIC_LAST_CLICKS',
              'METRIC_LAST_IMPRESSIONS',
              'METRIC_CM_POST_CLICK_REVENUE',
              'METRIC_CM_POST_VIEW_REVENUE',
              'METRIC_REVENUE_ADVERTISER'
            ],
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_MEDIA_PLAN',
              'FILTER_INSERTION_ORDER',
              'FILTER_LINE_ITEM',
              'FILTER_PAGE_LAYOUT',
              'FILTER_WEEK',
              'FILTER_MONTH',
              'FILTER_YEAR',
              'FILTER_PARTNER',
              'FILTER_LINE_ITEM_TYPE',
              'FILTER_DEVICE_TYPE',
              'FILTER_BROWSER'
            ]
          }
        }
      },
      'delete': False
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'account': {'field': {'order': 2,'name': 'cm_account_id','default': '','kind': 'string','description': 'Campaign Manager Account Id.'}},
        'timeout': 90,
        'body': {
          'delivery': {
            'emailOwner': False
          },
          'name': {'field': {'prefix': 'ITP_Audit_Floodlight_','name': 'recipe_name','order': 1,'kind': 'string','description': 'Name of report in DV360, should be unique.'}},
          'format': 'CSV',
          'type': 'FLOODLIGHT',
          'schedule': {
            'active': True,
            'every': 1,
            'repeats': 'WEEKLY',
            'repeatsOnWeekDays': [
              'Sunday'
            ]
          },
          'floodlightCriteria': {
            'floodlightConfigId': {
              'dimensionName': 'dfa:floodlightConfigId',
              'value': {'field': {'order': 4,'name': 'floodlight_configuration_id','default': '','kind': 'integer','description': 'Floodlight Configuration Id for the Campaign Manager floodlight report.'}},
              'kind': 'dfareporting#dimensionValue',
              'matchType': 'EXACT'
            },
            'reportProperties': {
              'includeUnattributedIPConversions': False,
              'includeUnattributedCookieConversions': True
            },
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_30_DAYS'
            },
            'dimensions': [
              {
                'name': 'dfa:site',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:floodlightAttributionType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:interactionType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:pathType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:browserPlatform',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:platformType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:week',
                'kind': 'dfareporting#sortedDimension'
              }
            ],
            'metricNames': [
              'dfa:activityClickThroughConversions',
              'dfa:activityViewThroughConversions',
              'dfa:totalConversions',
              'dfa:totalConversionsRevenue'
            ]
          },
          'kind': 'dfareporting#report'
        }
      },
      'out': {
        'bigquery': {
          'is_incremental_load': False,
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Floodlight_CM_Report'
        }
      },
      'delete': False
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'filters': {
          'dfa:advertiser': {
            'values': {'field': {'order': 3,'name': 'cm_advertiser_ids','default': '','kind': 'integer_list','description': 'Optional: Comma delimited list of CM advertiser ids.'}}
          }
        },
        'account': {'field': {'order': 2,'name': 'cm_account_id','default': '','kind': 'string','description': 'Campaign Manager Account Id.'}},
        'timeout': 90,
        'body': {
          'delivery': {
            'emailOwner': False
          },
          'name': {'field': {'name': 'recipe_name','description': 'Name of the Campaign Manager browser report.','kind': 'string','prefix': 'ITP_Audit_Browser_','order': 1,'default': 'ITP_Audit_Dashboard_Browser'}},
          'format': 'CSV',
          'type': 'STANDARD',
          'schedule': {
            'active': True,
            'every': 1,
            'repeats': 'WEEKLY',
            'repeatsOnWeekDays': [
              'Sunday'
            ]
          },
          'criteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_365_DAYS'
            },
            'metricNames': [
              'dfa:impressions',
              'dfa:clicks',
              'dfa:totalConversions',
              'dfa:activityViewThroughConversions',
              'dfa:activityClickThroughConversions'
            ],
            'dimensions': [
              {
                'name': 'dfa:campaign',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:campaignId',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:site',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:advertiser',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:advertiserId',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:browserPlatform',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:platformType',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:month',
                'kind': 'dfareporting#sortedDimension'
              },
              {
                'name': 'dfa:week',
                'kind': 'dfareporting#sortedDimension'
              }
            ],
            'dimensionFilters': [
            ]
          },
          'kind': 'dfareporting#report',
          'fileName': {'field': {'name': 'recipe_name','description': 'Name of the Campaign Manager browser report.','kind': 'string','prefix': 'ITP_Audit_Browser_','order': 1,'default': 'ITP_Audit_Dashboard_Browser'}}
        }
      },
      'out': {
        'bigquery': {
          'is_incremental_load': False,
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_CM_Browser_Report_Dirty'
        }
      },
      'delete': False
    }
  },
  {
    'sheets': {
      'range': 'A:B',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Environment'
        }
      },
      'tab': 'Enviroment',
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
    }
  },
  {
    'sheets': {
      'range': 'A:C',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Browser'
        }
      },
      'tab': 'Browser',
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
    }
  },
  {
    'sheets': {
      'range': 'A:C',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_CM_Browser_lookup'
        }
      },
      'tab': 'CM_Browser_lookup',
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
    }
  },
  {
    'sheets': {
      'range': 'A:B',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Device_Type'
        }
      },
      'tab': 'Device_Type',
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
    }
  },
  {
    'sheets': {
      'range': 'A:B',
      'auth': 'user',
      'header': True,
      'out': {
        'auth': 'user',
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Floodlight_Attribution'
        }
      },
      'tab': 'Floodlight_Attribution',
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'prefix': 'ITP_Audit_Browser_','name': 'recipe_name','order': 1,'kind': 'string','description': 'Name of report in DV360, should be unique.'}}
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
          'table': 'z_Dv360_Browser_Report_Dirty'
        }
      }
    }
  },
  {
    'itp_audit': {
      'dataset': {'field': {'order': 1,'name': 'recipe_slug','default': 'ITP_Audit_Dashboard','kind': 'string','description': 'BigQuery dataset for store dashboard tables.'}},
      'auth': 'user',
      'account': {'field': {'order': 2,'name': 'cm_account_id','default': '','kind': 'string','description': 'Campaign Manager Account Id.'}},
      'sheet': {'field': {'name': 'recipe_name','description': 'Name of document to deploy to.','kind': 'string','order': 1,'prefix': 'ITP Audit ','default': ''}},
      'timeout': 60
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
